Data description

In [1]:
from JoinDatasets import *
main_dir = "./data"
output_dir = "./merged-data"
join_data_sets(main_dir, output_dir)

Data relations

Errors and noise

Data difficulty

Data representation

Data normalisation

Data augmentation

Data splits